<a href="https://massstreetuniversity.com/"><img src="http://tutorials.massstreetuniversity.com/images/logo.png" alt="School Logo"></a><br><br><br>
<a href="http://tutorials.massstreetuniversity.com/transact-sql/">Tutorial Home</a>

<h1>How To Pivot Data With T-SQL When Columns Are Not Predefined</h1>

The PIVOT function is great. Until you find out that the function demands to know the column names prior to running the query. 

WACK! That's wack. 

When does that even happen in the real world?! Never. That's not how this works. That's not how any of this works.

Fortunately, there is a solution that utilizes some dynamic SQL and the deep magic.

<strong>Examples</strong>

So picture this exchange.

Accountant: Hey Bob, I need some data.

Me: Yo dawg! I'm your hook up! I'm about to HOOK you UP! What you want what you need dawg?

Accountant: I need to see the sales of all salespeople by month from the beginning of time.

Me: (*sucks teeth*) Man dawg that ain't nothing! I got you fam!

Accountant: Ok but I need to see it with people in rows and months as column headers.

Me: .....For real dawg?!

Unfortunately, this story is only partially made up. I don't really talk like this, but the request for the shape of his data totally happened. 

But hey. This is business analytics. We're ALL about some customer service. So we gives the people what they wants!

A lot of this code is magic that I can't explain. I just keep this query handy and alter it every time I need to do this. The key thing is to identify which column from your source data is going to wind up as column headers. The rest is just hitting F5 and praying.

In [ ]:
USE AdventureWorks2016

DECLARE @cols AS NVARCHAR(MAX)
DECLARE @query  AS NVARCHAR(MAX)

DROP TABLE IF EXISTS #SalesReport

CREATE TABLE #SalesReport ([SalesPerson] NVARCHAR(50), [MonthOfSale] TINYINT, [TotalSales] MONEY)
;


INSERT INTO #SalesReport ([SalesPerson], [MonthOfSale], [TotalSales])
SELECT
CONCAT(p.FirstName, ' ',p.LastName) AS SalesPerson,
DATEPART(month,soh.OrderDate) AS MonthOfSale,
SUM(sod.LineTotal) AS TotalSales
FROM Person.Person p
JOIN [Sales].[SalesPerson] sp
ON p.BusinessEntityID = sp.BusinessEntityID
JOIN [Sales].[SalesOrderHeader] soh
ON sp.BusinessEntityID = soh.SalesPersonID
JOIN [Sales].[SalesOrderDetail] sod
ON soh.SalesOrderID = sod.SalesOrderDetailID
GROUP BY p.LastName, p.FirstName, DATEPART(month,soh.OrderDate)
ORDER BY p.LastName, p.FirstName, DATEPART(month,soh.OrderDate)
;


select @cols = STUFF((SELECT distinct ',' + QUOTENAME([MonthOfSale]) 
                    from #SalesReport
            FOR XML PATH(''), TYPE
            ).value('.', 'NVARCHAR(MAX)') 
        ,1,1,'')

set @query = 'SELECT [SalesPerson], ' + @cols + ' from 
             (
                select [SalesPerson], [MonthOfSale], [TotalSales]
                from #SalesReport
            ) x
            pivot 
            (
                min([TotalSales])
                for [MonthOfSale] in (' + @cols + ')
            ) p '

execute(@query)

DROP TABLE #SalesReport

Copyright © 2020, Mass Street Analytics, LLC. All Rights Reserved.